# SQL Interview Preparation Notebook
This notebook contains a series of SQL exercises to help you prepare for your data engineering interview at Skyworks Solutions. The questions are designed using the Chinook dataset and will cover various SQL topics, including window functions, aggregations, and common interview scenarios.

Attempt each exercise by filling in the blank code cells provided.

## Question 1: Window Function - Row Number
Write a query to rank customers based on their total purchases. Use the `ROW_NUMBER()` window function to assign a rank to each customer partitioned by country and ordered by their `customer_country_total`. The sample data is shown below:
```
CustomerId | Country         | customer_country_total | rank_col
-----------|-----------------|------------------------|---------
1          | USA             | 100.00                 | 1
2          | USA             | 80.00                  | 2
...
```
Complete the following query:

In [ ]:
select 
    c.CustomerId, 
    c.Country, 
    SUM(i.Total) AS customer_country_total, 
    ROW_NUMBER() OVER(PARTITION BY c.Country ORDER BY SUM(i.Total) DESC) AS rank_col
from 
    Customer c 
    join Invoice i ON i.CustomerId = c.CustomerId
group by 
    c.CustomerId, 
    c.Country;

## Question 2: Window Function - RANK and DENSE_RANK
1. Use the `RANK()` window function to rank tracks by their `Milliseconds` within each album.
2. Use the `DENSE_RANK()` window function to achieve the same result and note the differences.
Use the `Track` table for this exercise.

In [ ]:
select  a.Title, t.Name, t.Milliseconds, RANK() OVER(partition by a.AlbumId ORDER BY t.Milliseconds DESC) as rank
from Track t 
join Album a on a.AlbumId = t.AlbumId
order by a.AlbumId, rank

In [ ]:
select  a.Title, t.Name, t.Milliseconds, DENSE_RANK() OVER(partition by a.AlbumId ORDER BY t.Milliseconds DESC) as rank
from Track t 
join Album a on a.AlbumId = t.AlbumId
order by a.AlbumId, rank

## Question 3: Window Function - NTILE
Divide customers into quartiles based on their total purchases using the `NTILE()` window function. Display the customer ID, country, total purchases, and their respective quartile.


In [ ]:
select 
    c.CustomerId, 
    c.Country, 
    SUM(i.Total) AS total_purchases,
    NTILE(4) OVER (ORDER BY SUM(i.Total) DESC) AS purchase_quartile
from 
    Customer c 
    join Invoice i ON i.CustomerId = c.CustomerId
group by 
    c.CustomerId, c.Country;

## Question 4: Window Function - LEAD and LAG
1. Use the `LEAD()` function to find the next track's name for each track in the `Track` table ordered by `TrackId`.
2. Use the `LAG()` function to find the previous track's name for each track in the same table and order.

In [ ]:
select t.TrackId, t.Name, LEAD(t.Name) OVER(ORDER BY t.TrackId)
from Track t

In [ ]:
select t.TrackId, t.Name, LAG(t.Name) OVER(ORDER BY t.TrackId)
from Track t

## Question 5: Aggregation with GROUP BY
Write a query to find the total sales amount for each country in the `Invoice` table. Sort the results by the total amount in descending order.


In [ ]:
select c.Country, SUM(i.Total) as country_total
from Invoice i join Customer C on i.CustomerId = c.CustomerId
group by c.Country

## Question 6: Complex Subqueries
Write a query to find customers who have spent more than the average amount spent by all customers. Use a subquery to find the average total purchase amount.


In [ ]:
select SUM(i.Total) as total_spent_outer
from Customer c join Invoice i on i.CustomerId = c.CustomerId
group by c.CustomerId
having
sum(i.Total) > (
    select AVG(total_spent)
    from (
        select SUM(i.Total) as total_spent
        from Invoice i 
        group by i.CustomerId
    )
)

## Question 7: JOINs with Aggregation
Using the `Invoice` and `Customer` tables, find the total amount spent by each customer along with their country. Sort the result by the total amount in descending order.


In [ ]:
select 
    c.customerid, 
    sum(i.total) as customer_total, 
    c.country,
    sum(sum(i.total)) over (partition by c.country) as country_total
from 
    customer c 
    join invoice i on i.customerid = c.customerid
group by 
    c.customerid, c.country
order by 
    customer_total desc;

## Question 8: Window Function - SUM with PARTITION
Use the `SUM()` window function to find the running total of sales for each country in the `Invoice` table, partitioned by `BillingCountry` and ordered by `InvoiceDate`.


In [ ]:
select i.InvoiceId, SUM(i.Total) OVER(partition by i.BillingCountry ORDER BY i.InvoiceDate) as running_total,
i.BillingCountry
from Invoice i

## Question 9: CASE Statements with Aggregation
Write a query that categorizes customers based on their total purchases: 'High Spender' for those spending over 100, 'Moderate Spender' for 50-100, and 'Low Spender' for less than 50. Use a `CASE` statement within a `SELECT` clause.


In [ ]:
select c.CustomerId, SUM(i.Total) as customer_total, 
CASE WHEN SUM(i.Total) > 100 THEN "High Spender"
    WHEN 50 <= SUM(i.Total) AND SUM(i.Total) <= 100 THEN "Moderate Spender"
    WHEN SUM(i.Total) < 50 THEN "Low Spender"
END as spending_category

FROM Invoice i join Customer c on i.CustomerId = c.CustomerId
GROUP BY c.CustomerId

## Question 10: Self JOINs
Using the `Employee` table, get a list of employees and the name of the manager they report to 


In [ ]:
select e.EmployeeId, e.FirstName AS employee_name, m.FirstName AS manager_name
from Employee e join Employee m ON e.ReportsTo = m.EmployeeId

Note there is no window function to do this b/c window functions don't understand heirachical relationships